In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
#loading in the csv data
df = pd.read_csv('/kaggle/input/fashion-product-images-small/myntradataset/styles.csv',error_bad_lines=False)

df.head()

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [4]:
df = df.dropna()
df.nunique()
df.columns

Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName'],
      dtype='object')

In [5]:
# Looking at all the unique labels in all categorical columns 
cat_columns = ['gender', 'masterCategory', 'subCategory', 'articleType','baseColour', 'season', 'year', 'usage']

for col in cat_columns:
    print(col)
    print(df[col].unique())
    print('-------------------------')

gender
['Men' 'Women' 'Boys' 'Girls' 'Unisex']
-------------------------
masterCategory
['Apparel' 'Accessories' 'Footwear' 'Personal Care' 'Free Items'
 'Sporting Goods' 'Home']
-------------------------
subCategory
['Topwear' 'Bottomwear' 'Watches' 'Socks' 'Shoes' 'Belts' 'Flip Flops'
 'Bags' 'Innerwear' 'Sandal' 'Shoe Accessories' 'Fragrance' 'Jewellery'
 'Lips' 'Saree' 'Eyewear' 'Scarves' 'Dress' 'Loungewear and Nightwear'
 'Wallets' 'Apparel Set' 'Headwear' 'Mufflers' 'Skin Care' 'Makeup'
 'Free Gifts' 'Ties' 'Accessories' 'Nails' 'Beauty Accessories'
 'Water Bottle' 'Skin' 'Eyes' 'Bath and Body' 'Gloves'
 'Sports Accessories' 'Cufflinks' 'Sports Equipment' 'Stoles' 'Hair'
 'Perfumes' 'Home Furnishing' 'Umbrellas' 'Wristbands' 'Vouchers']
-------------------------
articleType
['Shirts' 'Jeans' 'Watches' 'Track Pants' 'Tshirts' 'Socks' 'Casual Shoes'
 'Belts' 'Flip Flops' 'Handbags' 'Tops' 'Bra' 'Sandals' 'Shoe Accessories'
 'Sweatshirts' 'Deodorant' 'Formal Shoes' 'Bracelet' 'Lips

The images in this Dataset are very low resolution (80x60). We will be using the categories that are visually distinct even at such a low resolution.

The categories year, usage, season, and gender mighht not be clearly visually distinct in some cases, so we wont be using them.

The categories, masterCategory and subCategory are distinct enough groups, but they are not specific enough for practical use. 

The categories we will use are articleType and baseColour(You can use more categories if you want).

There are many unique labels in these categories, we will only be using the ones with more than 1000 examples, since we would need a good number of samples for proper classifcation.

In [6]:
value_counts = df['articleType'].value_counts()

indexes = value_counts.index

values = value_counts.values

for i in range(len(value_counts)):

    if values[i] <1000:
        break

types_used = indexes[:i]
print('Article types used: ',types_used)

Article types used:  Index(['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes',
       'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses'],
      dtype='object')


In [7]:
value_counts = df['baseColour'].value_counts()

indexes = value_counts.index

values = value_counts.values

for i in range(len(value_counts)):

    if values[i] <1000:
        break

colours_used = indexes[:i]
print('Base Colours used: ',colours_used)

Base Colours used:  Index(['Black', 'White', 'Blue', 'Brown', 'Grey', 'Red', 'Green', 'Pink',
       'Navy Blue', 'Purple', 'Silver'],
      dtype='object')


In [9]:
# Removing all the examples with labels other than the selected ones
 
df = df[df['articleType'].isin(types_used)]
df = df[df['baseColour'].isin(colours_used)]

In [10]:
#number of examples we are left with
len(df)

21835

Now we will load in all the images from the remaining rows, and convert them to numpy arrays with img_to_array function in keras.

In [14]:
data = []

# Reading all the images and processing the data in them 

from tensorflow.keras.preprocessing.image import img_to_array
import cv2

IX = 80
IY = 60

invalid_ids = []

for name in df.id:

    try:
        image = cv2.imread('/kaggle/input/fashion-product-images-small/myntradataset/images/'+str(name)+'.jpg')
        image = cv2.resize(image, (IX,IY) )
        image = img_to_array(image)
        data.append(image)        
    except: 
        # Images for certain ids are missing, so they are not added to the dataset  
        invalid_ids.append(name)

In [15]:
# ids of missing images
print('invalid ids:')
print(invalid_ids)

invalid ids:
[39403, 39425]


In [16]:
labels = []

used_columns = ['subCategory','baseColour']

# getting labels for the columns used

for index, row in df.iterrows():

    if row['id'] in invalid_ids:
        continue

    tags = []

    for col in used_columns:
        tags.append(row[col])

    labels.append(tags)

In [17]:
import numpy as np

# converting data into numpy arrays

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

print(labels)

[['Topwear' 'Navy Blue']
 ['Watches' 'Silver']
 ['Topwear' 'Grey']
 ...
 ['Shoes' 'White']
 ['Topwear' 'Blue']
 ['Watches' 'Pink']]


Now we will create binary vectors as the outputs of the model

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

# creating a binary vector for the input labels 

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print(mlb.classes_)
print(labels[0])

['Bags' 'Belts' 'Black' 'Blue' 'Brown' 'Eyewear' 'Free Gifts' 'Green'
 'Grey' 'Navy Blue' 'Pink' 'Purple' 'Red' 'Shoes' 'Silver' 'Topwear'
 'Watches' 'White']
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]


In [19]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

inputShape = (IY, IX, 3)

# A very simple sequential model is used since the images are very low resolution and the categories are fiarly distinct

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten()) 

model.add(Dense(128))
model.add(Activation('sigmoid'))


out = len(mlb.classes_)

model.add(Dense(out))
model.add(Activation('sigmoid')) # activation function for the final layer has to be sigmoid, since mutiple output labels can have value 1
                    
model.compile(loss='binary_crossentropy', # loss function has to be binary_crossentropy, it is calculated seperately for each of the outputs
              optimizer='adam',
              metrics=['mse'])

In [20]:
from sklearn.model_selection import train_test_split

# splitting data into testing and training set 

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.1, random_state=42)

In [21]:
batch = 32
E = 50

#training the model 
model.fit(x=trainX,y=trainY,
          epochs=E ,verbose=1)

Epoch 1/50
615/615 [==============================] - 6s 5ms/step - loss: 0.2811 - mse: 0.0814
Epoch 2/50
615/615 [==============================] - 3s 5ms/step - loss: 0.1118 - mse: 0.0308
Epoch 3/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0897 - mse: 0.0248
Epoch 4/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0813 - mse: 0.0226
Epoch 5/50
615/615 [==============================] - 3s 6ms/step - loss: 0.0759 - mse: 0.0211
Epoch 6/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0709 - mse: 0.0199
Epoch 7/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0706 - mse: 0.0199
Epoch 8/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0643 - mse: 0.0181
Epoch 9/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0628 - mse: 0.0177
Epoch 10/50
615/615 [==============================] - 3s 5ms/step - loss: 0.0600 - mse: 0.0169
Epoch 11/50
615/615 [============================

In [22]:
preds = model.predict(testX)


# since the predictions of the model are sigmoid, we will first binarize them to 0 or 1
pred_binarized = []

for pred in preds:
    vals = []
    for val in pred:
        if val > 0.5:
            vals.append(1)
        else:
            vals.append(0)
    pred_binarized.append(vals) 

pred_binarized = np.array(pred_binarized)   


# we convert the output vectors to the predicted labels
true_test_labels = mlb.inverse_transform(testY)
pred_test_labels = mlb.inverse_transform(pred_binarized)

correct = 0
wrong = 0

# Evaluating the predictions of the model

for i in range(len(testY)):

    true_labels = list(true_test_labels[i])

    pred_labels = list(pred_test_labels[i])

    label1 = true_labels[0]
    label2 = true_labels[1]

    if label1 in pred_labels:
        correct+=1
    else:
        wrong+=1

    if label2 in pred_labels:
        correct+=1
    else:
        wrong+=1    



print('correct: ', correct)
print('missing/wrong: ', wrong)
print('Accuracy: ',correct/(correct+wrong))

correct:  3696
missing/wrong:  672
Accuracy:  0.8461538461538461


We can see that this model identifies 84.61% of the labels correctly, let us see what that looks like in practice

In [23]:
for i in range(20):
    print('True labels: ',true_test_labels[i],' Predicted labels: ',pred_test_labels[i])

True labels:  ('Topwear', 'White')  Predicted labels:  ('Topwear', 'White')
True labels:  ('Black', 'Shoes')  Predicted labels:  ('Blue', 'Shoes')
True labels:  ('Green', 'Topwear')  Predicted labels:  ('Green', 'Topwear')
True labels:  ('Purple', 'Topwear')  Predicted labels:  ('Pink', 'Topwear')
True labels:  ('Green', 'Topwear')  Predicted labels:  ('Green', 'Topwear')
True labels:  ('Shoes', 'White')  Predicted labels:  ('Shoes', 'White')
True labels:  ('Brown', 'Shoes')  Predicted labels:  ('Brown', 'Shoes')
True labels:  ('Black', 'Watches')  Predicted labels:  ('Black', 'Watches')
True labels:  ('Black', 'Watches')  Predicted labels:  ('Black', 'Watches')
True labels:  ('Red', 'Topwear')  Predicted labels:  ('Brown', 'Topwear')
True labels:  ('Brown', 'Shoes')  Predicted labels:  ('Brown', 'Shoes')
True labels:  ('Black', 'Shoes')  Predicted labels:  ('Black', 'Shoes')
True labels:  ('Bags', 'Brown')  Predicted labels:  ('Bags', 'Brown')
True labels:  ('Shoes', 'White')  Predict

While we did not classify the images into based on all the category classes, we were able to classify them into more than one labels at the same time. 